# Tugas 2 Normalisasi text

1. Punctuation process menghilangkan tanda baca dan simbol

2. Stopword

3. Tokenisasi

4. Steeming

5. Feature extraction dan membentuk VSM dalam term frequency, logarithm freqency, one-hot encoding,  TF-IDF

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#install library
!pip install sastrawi
!pip install swifter
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=32123a93b3674f83368ad0c76126410cdb127656069a0e01a0096b78f37fbd90
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [3]:
#import library
import pandas as pd
import numpy as np

In [4]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
#load data
df = pd.read_csv("/content/drive/MyDrive/ppw/tugas/tugas-ppw/data.csv")
df.head()

,NIM,Judul,Abstrak,Program Studi,Penulis,Dosen Pembimbing 1,Dosen Pembimbing 2
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,Penulis : A.Ubaidillah S.Kom,Dosen Pembimbing I : Budi Setyono M.T,Dosen Pembimbing II :Hermawan S.T
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,"Penulis : M. Basith Ardianto,","Dosen Pembimbing I : Drs. Budi Soesilo, MT","Dosen Pembimbing II :Koko Joni, ST"
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\nEN...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,"Penulis : Akhmad Suyandi, S.Kom","Dosen Pembimbing I : Drs. Budi Soesilo, M.T","Dosen Pembimbing II :Hermawan, ST, MT"
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,Penulis : Heri Supriyanto,"Dosen Pembimbing I : Mulaab, S.Si., M.Kom","Dosen Pembimbing II :Firli Irhamni, ST., M.Kom"
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,Penulis : Septian Rahman Hakim,"Dosen Pembimbing I : Arik Kurniawati, S.Kom., ...","Dosen Pembimbing II :Haryanto, S.T., M.T."


In [6]:
df.shape

(858, 7)

## Menghilangkan Tanda Baca

In [7]:
#Remove Puncutuation
clean_tag = re.compile('@\S+')
clean_url = re.compile('https?:\/\/.*[\r\n]*')
clean_hastag = re.compile('#\S+')
clean_symbol = re.compile('[^a-zA-Z]')
def clean_punct(text):
    text = clean_tag.sub('', str(text))
    text = clean_url.sub('', text)
    text = clean_hastag.sub(' ', text)
    text = clean_symbol.sub(' ', text)
    return text
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
preprocessing = df['Abstrak'].apply(clean_punct)
clean=pd.DataFrame(preprocessing)
clean

,Abstrak
0,Sistem informasi akademik SIAKAD merupaka...
1,Berjalannya koneksi jaringan komputer dengan l...
2,Web server adalah sebuah perangkat lunak serve...
3,Penjadwalan kuliah di Perguruan Tinggi me...
4,Seiring perkembangan teknologi yang ada diduni...
...,...
853,Investasi saham selama ini memiliki resiko ker...
854,Information Retrieval IR merupakan pengambil...
855,Klasifikasi citra merupakan proses pengelompok...
856,Identifikasi atribut pejalan kaki merupakan sa...


## Tokenisasi

In [8]:
data_clean=[]
for i in range(len(preprocessing)):
  data_clean.append(preprocessing[i])

In [9]:
tokenize=[]
for i in range(len(data_clean)):
  token=word_tokenize(data_clean[i])
  tokendata = []
  for x in token :
    tokendata.append(x)
  tokenize.append(tokendata)
  print(tokendata)

['Sistem', 'informasi', 'akademik', 'SIAKAD', 'merupakan', 'sistem', 'informasi', 'yang', 'berfungsi', 'menangani', 'pengelolaan', 'dan', 'penyajian', 'data', 'data', 'akademik', 'yang', 'oleh', 'pihak', 'fakultas', 'SIAKAD', 'dianggap', 'sangat', 'penting', 'dalam', 'memberikan', 'pelayanan', 'mahasiswa', 'yang', 'membutuhkan', 'informasi', 'akademik', 'Di', 'Universitas', 'Trunojoyo', 'telah', 'tersedia', 'SIAKAD', 'namun', 'masih', 'menggunakan', 'database', 'terpusat', 'Sistem', 'seperti', 'ini', 'memberikan', 'kelebihan', 'yaitu', 'perawatannya', 'mudah', 'selain', 'itu', 'juga', 'membutuhkan', 'sedikit', 'biaya', 'namun', 'sistem', 'tersebut', 'juga', 'berpotensi', 'mengahadapi', 'kendala', 'kendala', 'yaitu', 'dalam', 'proses', 'transaksi', 'data', 'karena', 'padatnya', 'jaringan', 'yang', 'menuju', 'database', 'SIAKAD', 'kelambatan', 'dalam', 'pemrosesan', 'respon', 'query', 'dikarenakan', 'data', 'yang', 'tersimpan', 'semakin', 'besar', 'dan', 'pemrosesan', 'semakin', 'komplek

## Stopwords Removal

In [10]:
stopword=[]
for i in range(len(tokenize)):
  listStopword =  set(stopwords.words('indonesian'))
  removed=[]
  for x in (tokenize[i]):
    if x not in listStopword:
       removed.append(x)
  stopword.append(removed)
  print(removed)

['Sistem', 'informasi', 'akademik', 'SIAKAD', 'sistem', 'informasi', 'berfungsi', 'menangani', 'pengelolaan', 'penyajian', 'data', 'data', 'akademik', 'fakultas', 'SIAKAD', 'dianggap', 'pelayanan', 'mahasiswa', 'membutuhkan', 'informasi', 'akademik', 'Di', 'Universitas', 'Trunojoyo', 'tersedia', 'SIAKAD', 'database', 'terpusat', 'Sistem', 'kelebihan', 'perawatannya', 'mudah', 'membutuhkan', 'biaya', 'sistem', 'berpotensi', 'mengahadapi', 'kendala', 'kendala', 'proses', 'transaksi', 'data', 'padatnya', 'jaringan', 'database', 'SIAKAD', 'kelambatan', 'pemrosesan', 'respon', 'query', 'data', 'tersimpan', 'pemrosesan', 'kompleks', 'memiliki', 'kelemahan', 'ketersediaan', 'data', 'Untuk', 'sistem', 'pengembangan', 'sistem', 'database', 'sistem', 'databases', 'terdistribusi', 'fakultas', 'dijadikan', 'solusi', 'permasalahan', 'Karena', 'basisdata', 'terdistribusi', 'keuntungan', 'dimiliki', 'basisdata', 'terpusat', 'pengawasan', 'distribusi', 'reability', 'availability', 'kecepatan', 'pemros

## Merubah isi dari kolum abstrak

In [11]:
join=[]
for i in range(len(stopword)):
  joinkata = ' '.join(stopword[i])
  join.append(joinkata)

hasilpreproses = pd.DataFrame(join, columns=['Abstrak'])
hasilpreproses

,Abstrak
0,Sistem informasi akademik SIAKAD sistem inform...
1,Berjalannya koneksi jaringan komputer lancar g...
2,Web server perangkat lunak server berfungsi me...
3,Penjadwalan kuliah Perguruan Tinggi kompleks D...
4,Seiring perkembangan teknologi didunia muncul ...
...,...
853,Investasi saham memiliki resiko kerugian perge...
854,Information Retrieval IR pengambilan informasi...
855,Klasifikasi citra proses pengelompokan piksel ...
856,Identifikasi atribut pejalan kaki salah peneli...


## Term Frequency

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(np.array(hasilpreproses['Abstrak']))

In [14]:
tf = vectorizer.get_feature_names_out()
tf

array(['aalysis', 'aam', 'abad', ..., 'zoom', 'zucara', 'zungu'],
      dtype=object)

In [15]:
tf_array = X.toarray()
print(tf_array)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
df_tf= pd.DataFrame(tf_array, columns = tf)
df_tf

,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,abstract,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df_tf.to_excel("/content/drive/MyDrive/ppw/tugas/tugas-ppw/df_tf.xlsx")

## Logarithm frequency

Logarithm frequency adalah sebuah metode transformasi yang digunakan dalam analisis data untuk mengubah data frekuensi menjadi skala logaritma

$If_{i, j}=\left\{\begin{array}{ll}
In(tf_{i,j} + 1 ), & \text { if } t f_{i, j}>0 \\
0, & \text { otherwise }
\end{array}\right\}.$

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
log_vectorizer = TfidfVectorizer(use_idf=False, smooth_idf=False, norm=None)
X_log = log_vectorizer.fit_transform(hasilpreproses['Abstrak'].tolist())

log_terms = log_vectorizer.get_feature_names_out()
df_log = pd.DataFrame(data = X_log.toarray(),columns = log_terms)
df_log

,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,abstract,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df_log.to_excel("/content/drive/MyDrive/ppw/tugas/tugas-ppw/df_log.xlsx", index=False)

## Binary Frequency

One-hot encoding digunakan untuk mengubah data kategori atau kualitatif menjadi representasi biner yang dapat digunakan dalam perhitungan dan analisis statistik.


$n_{i, j}=\left\{\begin{array}{ll}
1, & \text { if } t f_{i, j}>0 \\
0, & \text { otherwise }
\end{array}\right\}.$

In [20]:
def binarize_dataframe(df):
    # Salin DataFrame asli untuk menghindari perubahan inplace
    df_copy = df.copy()

    # Loop melalui semua kolom
    for col in df_copy.columns:
        # Ganti nilai-nilai lebih dari 0 menjadi 1
        df_copy[col] = df_copy[col].apply(lambda x: 1 if x > 0 else 0)

    return df_copy

# Contoh penggunaan fungsi dengan DataFrame yang telah diperbarui sebelumnya
binary_data = binarize_dataframe(df_tf)
binary_data

,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,abstract,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
855,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
856,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
binary_data.to_excel("/content/drive/MyDrive/ppw/tugas/tugas-ppw/binary_data.xlsx", index=False)

## TF - IDF

TF-IDF adalah singkatan dari "Term Frequency-Inverse Document Frequency" dalam pemrosesan bahasa alami dan analisis teks. Ini adalah metode statistik yang digunakan untuk mengukur pentingnya sebuah kata atau istilah dalam suatu dokumen terhadap koleksi dokumen yang lebih besar.

In [22]:
tr_idf_model  = TfidfVectorizer()
tf_idf_vector = tr_idf_model.fit_transform(hasilpreproses['Abstrak'])

In [23]:
tf_idf_array = tf_idf_vector.toarray()
print(tf_idf_array)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [24]:
words_set = tr_idf_model.get_feature_names_out()
print(words_set)

['aalysis' 'aam' 'abad' ... 'zoom' 'zucara' 'zungu']


In [25]:
df_tf_idf = pd.DataFrame(tf_idf_array, columns = words_set)
df_tf_idf

,aalysis,aam,abad,abadi,ability,abjad,absensi,absolut,absolute,abstract,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_tf_idf.to_excel("/content/drive/MyDrive/ppw/tugas/tugas-ppw/df_tf_idf.xlsx", index=False)